In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
data = pd.read_csv('林智堅抄襲.csv')

In [2]:
data_start = min(data['timestamp'])
data_end = max(data['timestamp'])

In [3]:
from datetime import datetime
data_start_str = datetime.fromtimestamp(int(float(data_start)/1000))
data_end_str = datetime.fromtimestamp(int(float(data_end)/1000))
print('{} ~ {}'.format(data_start_str, data_end_str))
print('news count: {}'.format(len(data)))

2022-07-05 09:35:00 ~ 2022-08-21 18:50:57
news count: 541


In [4]:
from datetime import datetime
timestamps = []
for item in data['timestamp'].to_list():
    t = datetime.fromtimestamp(int(item/1000))
    date_time = t.strftime("%Y%m%d000000")
    timestamps.append(date_time)
data['timestamp'] = timestamps

In [5]:
from sentence_transformers import SentenceTransformer

/home/ricky42613/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = SentenceTransformer('ckiplab/albert-base-chinese')

No sentence-transformers model found with name /home/ricky42613/.cache/torch/sentence_transformers/ckiplab_albert-base-chinese. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ricky42613/.cache/torch/sentence_transformers/ckiplab_albert-base-chinese were not used when initializing AlbertModel: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of A

In [7]:
from bertopic import BERTopic

In [8]:
from tokenizer import ChineseTokenizer
tokenizer = ChineseTokenizer()

In [9]:
from tqdm import tqdm
toks = []
sents = data['sentence'].tolist()
for i in tqdm(range(len(sents))):
    s = sents[i]
    tok_list = list(tokenizer.tokenize(s, rm_noisy = False)[0])
    sent = ' '.join(tok_list)
    toks.append(sent)
    # print(i)
# sents_tok = tokenizer.tokenize(sents)
# for sent in sents_tok:
#     sent = ' '.join(sent)
#     toks.append(sent)
#     embed.append(model.encode(sent, show_progress_bar=False))
data['token_sent'] = toks
data.to_csv('林智堅抄襲_token.csv')
# data['embed'] = embed

 67%|██████▋   | 360/541 [00:52<00:18, 10.01it/s]

In [ ]:
from umap.umap_ import UMAP
embed = model.encode(toks, show_progress_bar=True).tolist()
umap_model = UMAP(n_neighbors=10, n_components=400, metric='cosine', min_dist=0.0, random_state=42)
# for i in tqdm(range(len(toks))):
#     sent = toks[i]
#     embed.append(model.encode(sent, show_progress_bar=False))

Batches: 100%|██████████| 17/17 [00:00<00:00, 31.52it/s]


In [ ]:
import hdbscan
hdbscan_parameter = {
    'min_cluster_size': 15,
    'min_samples': 1,
    'cluster_selection_method': 'leaf'
}
cluster_model = hdbscan.HDBSCAN(
    min_cluster_size=hdbscan_parameter['min_cluster_size'],
    min_samples=hdbscan_parameter['min_samples'], 
    cluster_selection_method=hdbscan_parameter['cluster_selection_method'], 
    )
topic_model = BERTopic(embedding_model=model, 
                        umap_model=umap_model, 
                        hdbscan_model=cluster_model,
                        )

In [ ]:
import numpy as np
topics, probs = topic_model.fit_transform(data['token_sent'], np.array(embed))

401
541


In [ ]:
keyword = []
for i in topics:
    keyword_list = [w[0] for w in topic_model.get_topic(i)[:3]]
    keyword.append(' '.join(keyword_list))

In [ ]:
data['topics'] = topics
# data['score'] = probs
data['keyword'] = keyword
data = data[data['topics'] != -1]
# data = data[data['score'] > 0.8]

In [ ]:
data.to_csv('林智堅抄襲_cluster.csv')

In [ ]:
fig = topic_model.visualize_hierarchy()

In [ ]:
fig.write_html("templates/hie.html")

In [ ]:
topics_over_time = topic_model.topics_over_time(data['token_sent'], data['topics'], data['timestamp'].to_list())

In [ ]:
fig = topic_model.visualize_topics_over_time(topics_over_time)
fig.write_html("templates/timeline.html")

# TimeLine Distance

In [ ]:
data = data.sort_values(by=['timestamp'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
embed = model.encode(toks, show_progress_bar=True).tolist()
distance = []
for i in tqdm(range(1, len(embed))):
    answerLikeliness = cosine_similarity([embed[i]], [embed[i-1]])
    score = answerLikeliness[0][0]
    distance.append(score)
    

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(distance)), distance)